## Prepare the Training data
---


We will create a new workflow that processes the data:

1. Remove too cloudy scenes
   * Check the ratio of the valid data for each patch and for each time frame
   * Keep only > 80 % valid coverage
2. Concatenate BAND, NDVI, NDWI, NDBI info into a single feature called FEATURES
3. As this example only covers one image taken on one day; ```Linear Interpolation``` is not done. For larger time periods it would be.
   * Perform temporal Interpolation
   * Create a task for linear interpolation in the temporal dimension
   * Provide the cloud mask to tell the interpolating function which values to update```
4. Perform erosion
   * This removes artefacts with a width of 1 px, and also removes the edges between polygons of different classes
5. Random spatial sampling of the EOPatches
   * Randomly take a subset of pixels from a patch to use in the machine learning training
6. Split patches for training/validation
   * Split the patches into a training and validation set

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
import sys
import os

import pickle

# Basics of Python data handling and visualization
import itertools
import numpy as np
np.random.seed(42)
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from IPython.display import Image 

from tqdm import tqdm_notebook as tqdm
from aenum import MultiValueEnum

In [ ]:
# Imports from eo-learn and sentinelhub-py
from eolearn.core import EOTask, EOPatch, LinearWorkflow, FeatureType, OverwritePermission, LoadTask, SaveTask, EOExecutor, ExtractBandsTask, MergeFeatureTask
from eolearn.geometry import VectorToRaster, PointSamplingTask, ErosionTask
from eolearn.features import LinearInterpolation, SimpleFilterTask, NormalizedDifferenceIndexTask
from sentinelhub import UtmZoneSplitter, BBox, CRS, DataSource

In [ ]:
# Machine learning 
import lightgbm as lgb
#import joblib
from sklearn.externals import joblib
from sklearn import metrics
from sklearn import preprocessing

In [ ]:
from pathlib import Path
path = Path('./')

In [ ]:
class ValidDataFractionPredicate:
    """
    Predicate that defines if a frame from EOPatch's time-series is valid or not. Frame is valid, if the 
    valid data fraction is above the specified threshold.
    """
    def __init__(self, threshold):
        self.threshold = threshold
        
    def __call__(self, array):
        coverage = np.sum(array.astype(np.uint8)) / np.prod(array.shape)
        return coverage > self.threshold

In [ ]:
path_out = './data/eopatches'

In [ ]:
# TASK TO LOAD EXISTING EOPATCHES
load = LoadTask(path_out)

# TASK FOR CONCATENATION
concatenate = MergeFeatureTask({FeatureType.DATA: ['BANDS', 'NDVI', 'NDWI', 'NDBI']},
                               (FeatureType.DATA, 'FEATURES'))

# TASK FOR FILTERING OUT TOO CLOUDY SCENES
# keep frames with > 80 % valid coverage
valid_data_predicate = ValidDataFractionPredicate(0.8)
filter_task = SimpleFilterTask((FeatureType.MASK, 'IS_VALID'), valid_data_predicate)

# TASK FOR LINEAR INTERPOLATION
# linear interpolation of full time-series and date resampling
resampled_range = ('2020-02-28', '2020-03-02', 3)
linear_interp = LinearInterpolation(
    'FEATURES', # name of field to interpolate
    mask_feature=(FeatureType.MASK, 'IS_VALID'), # mask to be used in interpolation
    copy_features=[(FeatureType.MASK_TIMELESS, 'LndC')], # features to keep
    resample_range=resampled_range, # set the resampling range
    bounds_error=False # extrapolate with NaN's
)

# TASK FOR EROSION
# erode each class of the reference map
erosion = ErosionTask(mask_feature=(FeatureType.MASK_TIMELESS,'LndC','LndC_ERODED'), disk_radius=1)

# TASK FOR SPATIAL SAMPLING
# Uniformly sample about pixels from patches
n_samples = 125000 # half of pixels
ref_labels = list(range(18)) # reference labels to take into account when sampling
spatial_sampling = PointSamplingTask(
    n_samples=n_samples, 
    ref_mask_feature='LndC_ERODED', 
    ref_labels=ref_labels, 
    sample_features=[  # tag fields to sample
        (FeatureType.DATA, 'FEATURES'),
        (FeatureType.MASK_TIMELESS, 'LndC_ERODED')
    ])

path_out_sampled = './data/eopatches_sampled'
if not os.path.isdir(path_out_sampled):
    os.makedirs(path_out_sampled)
save = SaveTask(path_out_sampled, overwrite_permission=OverwritePermission.OVERWRITE_PATCH)

In [ ]:
# Define the workflow
workflow = LinearWorkflow(
    load,
    concatenate,
    filter_task,
    #linear_interp,
    erosion,
    spatial_sampling,
    save
)

### Run the EOWorkflow over all EOPatches

In [ ]:
# Get the patchIDs from the previous notebook
#load array
fp = os.path.join(path/'data'/'tile-def/', 'patchIDs.csv')
patchIDs = np.fromfile(fp, dtype=int)
# print the array
print(patchIDs)

In [ ]:
%%time
   
execution_args = []
for idx in patchIDs:
    execution_args.append({
        load: {'eopatch_folder': f'eopatch_{idx}'},
        spatial_sampling: {'seed': 42},
        save: {'eopatch_folder': f'eopatch_{idx}'}
    })
    
executor = EOExecutor(workflow, execution_args, save_logs=False)
executor.run(workers=1, multiprocess=False)

executor.make_report()

## Model construction and training

The patches are split into a ```train``` and ```test``` subsets.

Because of the small area the ```test``` sample in hand picked. With a large dataset, the training and testing patches should be randomly chosen.

The sampled features and labels are loaded and reshaped into $n \times m$, where $n$ represents the number of training pixels, and $m = f \times t$ the number of all features, with $f$ the size of bands and band combinations (in this example 9) and $t$ the length of the resampled time-series (in this example 1)

[LightGBM](https://github.com/Microsoft/LightGBM) is used as the Machine Learning model taken directly from the [eo-learn](https://eo-learn.readthedocs.io/en/latest/examples/land-cover-map/SI_LULC_pipeline.html#6.-Model-construction-and-training) example. Near default hyper-parameters are used. [Parameter tuning](https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html) is possible and suggestions are welcome.

In [ ]:
# load sampled eopatches
eopatches = []
path_out_sampled = './data/eopatches_sampled'

for idx in patchIDs:
    eopatches.append(EOPatch.load(f'{path_out_sampled}/eopatch_{idx}', lazy_loading=True))    

eopatches = np.array(eopatches)

In [ ]:
print(eopatches.shape, eopatches.ndim)

In [ ]:
# Definition of the train and test patch IDs, take 80 % for train
#NB: it wants the index not the grid number
test_ID = [5, 8, 16, 22, 19]
train_ID = np.argwhere(~np.in1d(patchIDs, patchIDs[test_ID])).squeeze(axis=-1)

# Set the features and the labels for train and test sets
features_train = np.array([eopatch.data['FEATURES_SAMPLED'] for eopatch in eopatches[train_ID]])
labels_train = np.array([eopatch.mask_timeless['LndC_ERODED_SAMPLED'] for eopatch in eopatches[train_ID]])
features_test = np.array([eopatch.data['FEATURES_SAMPLED'] for eopatch in eopatches[test_ID]])
labels_test = np.array([eopatch.mask_timeless['LndC_ERODED_SAMPLED'] for eopatch in eopatches[test_ID]])

#get shape
p1, t, w, h, f = features_train.shape
p2, t, w, h, f = features_test.shape
p = p1 + p2

# reshape to n x m
features_train = np.moveaxis(features_train, 1, 3).reshape(p1 * w * h, t * f)
#features_train = np.reshape(p1 * w * h, t * f)
labels_train = np.moveaxis(labels_train, 1, 2).reshape(p1 * w * h, 1).squeeze()
#labels_train = np.reshape(p1 * w * h, 1).squeeze()
features_test = np.moveaxis(features_test, 1, 3).reshape(p2 * w * h, t * f)
#features_test = np.reshape(p2 * w * h, t * f)
labels_test = np.moveaxis(labels_test, 1, 2).reshape(p2 * w * h, 1).squeeze()
#labels_test = np.reshape(p2 * w * h, 1).squeeze()

# remove points with no reference from training (so we dont train to recognize "no data")
mask_train = labels_train == 0
features_train = features_train[~mask_train]
labels_train = labels_train[~mask_train]

# remove points with no reference from test (so we dont validate on "no data", which doesn't make sense)
mask_test = labels_test == 0
features_test = features_test[~mask_test]
labels_test = labels_test[~mask_test]

In [ ]:
#check tht test and training labels went through - remember there are 16 land cover classes
print(len(np.unique(labels_test)), len(np.unique(labels_test)))

In [ ]:
np.unique(labels_test)

In [ ]:
# check the shape of the features: in this example - the '9' represents the bands
# if we had a larger time period it would change the shape. The 9 would be greater.
features_train.shape

### Set up and train the model

In [ ]:
%%time
# Set up training classes
labels_unique = np.unique(labels_train)

# Set up the model
model = lgb.LGBMClassifier(boosting_type = 'dart',
    num_leaves = 50,
    objective ='multiclass', 
    learning_rate = 0.07,
    num_class =len(labels_unique), 
    metric = 'multi_logloss',
    random_state = 42,
    min_data_in_leaf = 1000,
    max_depth = 7,
    lambda_l1 =  0.5, # L1 regularization.
    lambda_l2 = 0.5, # L2 regularization.# stores validation results.
)

In [ ]:
# train the model
model.fit(features_train, labels_train)

# uncomment to save the model
joblib.dump(model, './model_SI_LndC.pkl')

## Validation

In order to validate the model, we use the training set to predict the classes, and then compare the predicted set of labels to the **"ground truth"**.

Our **"ground truth"** is a derivative of the 2018 South African National Land Cover; which is a 10-meter accurate product. 

As per remote sensing standards; validation is performed by evaluating metrics, such as accuracy, precision, recall, $F_1$ score. Nicely described [in this blog post](https://medium.com/greyatom/performance-metrics-for-classification-problems-in-machine-learning-part-i-b085d432082b).

In [ ]:
# load the model
model_path = './model_SI_LndC.pkl'
model = joblib.load(model_path)

# predict the test labels
plabels_test = model.predict(features_test)

Get the overall accuracy (OA) and the weighted $F_1$ score and the $F_1$ score, precision, and recall for each class separately

In [ ]:
#load the LandCover classes
class LndC(MultiValueEnum):
    """ 
    Enum class containing LandCover types
    """
    Woodland_and_Forest             = 'Woodland and Forest',  1, '#008000'
    Shrub_and_Grassland             = 'Shrub and Grassland)', 2, '#9370DB'
    Water                           = 'Water',                3, '#000080'
    Mines                           = 'Mines',                4, '#8B0000'
    Wetlands                        = 'Wetlands',             5, '#00CED1'
    Bare_Non_Vegetated              = 'Bare Non-Vegetated',   6, '#FFFACD'
    Cultivated_Commercial           = 'Cultivated_Commercial',7, '#DC143C'
    Fallow_land                     = 'Fallow land',          8, '#F08080'
    Formal_Residential              = 'Formal Residential',   9, '#FFA500'
    Informal_Residential            = 'Informal Residential', 10, '#FF69B4'
    Village                         = 'Village',              11, '#FF8C00'
    Smallholding                    = 'Smallholding',         12, '#DDA0DD'
    Urban_Recreation                = 'Urban Recreation',     13, '#7FFF00'
    Commercial                      = 'Commercial',           14, '#DAA520'
    Industrial                      = 'Industrial',           15, '#B8860B'
    Major_Road_and_Rail             = 'Major Road and Rail',  16, '#FFD700'
    
    @property
    def id(self):
        """ Returns an ID of an enum type
        :return: An ID
        :rtype: int
        """
        return self.values[1]

    @property
    def color(self):
        """ Returns class color
        :return: A color in hexadecimal representation
        :rtype: str
        """
        return self.values[2]

def get_bounds_from_ids(ids):
    bounds = []
    for i in range(len(ids)):
        if i < len(ids) - 1:
            if i == 0:
                diff = (ids[i + 1] - ids[i]) / 2
                bounds.append(ids[i] - diff)
            diff = (ids[i + 1] - ids[i]) / 2
            bounds.append(ids[i] + diff)
        else:
            diff = (ids[i] - ids[i - 1]) / 2
            bounds.append(ids[i] + diff)
    return bounds 

# Reference colormap things
lulc_bounds = get_bounds_from_ids([x.id for x in LndC])
lulc_cmap = ListedColormap([x.color for x in LndC], name="lulc_cmap")
lulc_norm = BoundaryNorm(lulc_bounds, lulc_cmap.N)

In [ ]:
class_labels = np.unique(labels_test)
class_names = np.array([entry.name for entry in LndC])
mask = np.in1d(plabels_test, labels_test)
pred = plabels_test[mask]
lbls = labels_test[mask]

f1_scores = metrics.f1_score(lbls, pred, labels=class_labels, average=None)
recall = metrics.recall_score(lbls, pred, labels=class_labels, average=None)
precision = metrics.precision_score(lbls, pred, labels=class_labels, average=None)

print('Classification accuracy {:.1f}%'.format(100 * metrics.accuracy_score(lbls, pred)))
print('Classification F1-score {:.1f}%'.format(100 * metrics.f1_score(lbls, pred, average='weighted')))
print()
print('             Class                   =  F1  | Recall  | Precision')
print('         --------------------------------------------------')
for idx in class_labels:
    print('         * {0:25s} = {1:4.1f} |  {2:4.1f}  | {3:3.1f}'.format(class_names[idx - 1], 
                                                                         f1_scores[idx - 1] * 100, 
                                                                         recall[idx - 1] * 100, 
                                                                         precision[idx - 1] * 100))

![image](figs/F1RecPrec.png)

In [ ]:
#write the printout to a csv via pandas
df = {'Labels' : pd.Series(class_labels), 
      'Classes' : pd.Series(class_names),      
      'F1' : pd.Series(f1_scores), 
      'Recall' : pd.Series(recall),
      'Precision' : pd.Series(precision)}
df = pd.DataFrame(df)

df.to_csv('./data/f1_recall_precision.csv', ';')

In [ ]:
df.head()

<div class="alert alert-block alert-info">
<b>STOP:</b> 

The results might not seem stella: but it could be down to the classes chosen. I should have merged Smallholding and Village. I could possibly have combined Fallow_land with Bare_Non-Vegetated as well (the Confusion Matrix below confirms this). Also its difficult for an algorithm to know the difference between Urban_Recreation and Grassland; which might appear similar. The same goes for the Industrial and Commercial classes. We're really at the boarder of "Land-Use" and Land-Cover. Iether way; its a good test.
    
```[Parameter Tuning]```(https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html) might also achieve a better result,
    
</div>

### Plot the standard and transposed Confusion Matrix

In [ ]:
# Define the plotting function
def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues, ylabel='True label', xlabel='Predicted label', filename=None):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    np.set_printoptions(precision=2, suppress=True)
    
    if normalize:
        cm = cm.astype('float') / (cm.sum(axis=1)[:, np.newaxis] + np.finfo(np.float).eps)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=1)
    plt.title(title, fontsize=11)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=9)
    plt.yticks(tick_marks, classes, fontsize=9)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black",
                 fontsize=8)

    plt.tight_layout()
    plt.ylabel(ylabel, fontsize=12)
    plt.xlabel(xlabel, fontsize=12)

In [ ]:
fig = plt.figure(figsize=(8, 8))

conf_matrix_gbm = metrics.confusion_matrix(lbls, pred)
plot_confusion_matrix(conf_matrix_gbm, 
                      classes=[class_names[idx - 1] for idx in class_labels], 
                      normalize=True, 
                      ylabel='Truth (LAND COVER)', 
                      xlabel='Predicted (GBM)',
                      title='Confusion matrix');

fig.savefig(f'figs/ConfusionMatrix.png', bbox_inches='tight')
plt.tight_layout()

![title](figs/ConfusionMatrix.png)

In [ ]:
fig = plt.figure(figsize=(8, 8))

conf_matrix_gbm = metrics.confusion_matrix(pred, lbls)
plot_confusion_matrix(conf_matrix_gbm, 
                      classes=[class_names[idx - 1] for idx in class_labels], 
                      normalize=True, 
                      xlabel='Truth (LAND COVER)', 
                      ylabel='Predicted (GBM)',
                      title='Transposed Confusion matrix');
fig.savefig(f'figs/Transposed ConfusionMatrix.png', bbox_inches='tight')
plt.tight_layout()

![title](figs/TransposedConfusionMatrix.png)

### How often do the classes appear in the dataset

In [ ]:
fig = plt.figure(figsize=(20, 5))

label_ids, label_counts = np.unique(labels_train, return_counts=True)

plt.bar(range(len(label_ids)), label_counts)
plt.xticks(range(len(label_ids)), [class_names[i -1] for i in label_ids], rotation=45, fontsize=15);
plt.yticks(fontsize=20);
fig.savefig(f'figs/labels_count.png', bbox_inches='tight')

![image](figs/label_count.png)

### AUC-ROC curves and  metrics

Calculate precision and recall rates, draw (Receiver Operating Characteristic) ROC curves  and calculate (Area Under The Curve) AUC. Also known as AUROC (Area Under the Receiver Operating Characteristics). ROC is a probability curve and AUC represents degree or measure of separability. 

AUC - ROC curve is a performance measurement for classification problems at various thresholds settings. It tells how much model is capable of distinguishing between classes. The reader is refered to [here](https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5).

In [ ]:
class_labels = np.unique(np.hstack([labels_test, labels_train]))

scores_test = model.predict_proba(features_test)
labels_binarized = preprocessing.label_binarize(labels_test, classes=class_labels)

fpr = dict()
tpr = dict()
roc_auc = dict()

for idx,lbl in enumerate(class_labels):
    fpr[idx -1], tpr[idx -1], _ = metrics.roc_curve(labels_binarized[:, idx], scores_test[:, idx])
    roc_auc[idx -1] = metrics.auc(fpr[idx -1], tpr[idx -1])   

In [ ]:
plt.figure(figsize=(20, 10))

for idx,lbl in enumerate(class_labels):
    if np.isnan(roc_auc[idx -1]):
        continue
    plt.plot(fpr[idx -1], tpr[idx -1], color=lulc_cmap.colors[lbl -1],
         lw=2, label=class_names[lbl -1] + ' (%0.5f)' % roc_auc[idx -1])
    

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 0.99])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=20)
plt.ylabel('True Positive Rate', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.title('ROC Curve', fontsize=20)
plt.legend(loc="lower right", prop={'size': 13})
fig.savefig(f'figs/roc_curve.png', bbox_inches='tight')
plt.show()

![image](figs/roc_curve.png)

### Most important features

Let us now check which features are most important in the above classification. The LightGBM model already contains the information about feature importances, so we only need to query them. This functionality would be more effective with a larger time-series. 

In [ ]:
# names of features
fnames = ['B2','B3','B4','B8','B11','B12','NDVI','NDWI','NDBI']

# get feature importances and reshape them to dates and features
feature_importances = model.feature_importances_.reshape((t, f))

fig = plt.figure(figsize=(15, 15))
ax = plt.gca()

# plot the importances
im = ax.imshow(feature_importances, aspect=0.25)
plt.xticks(range(len(fnames)), fnames, rotation=45, fontsize=20)
plt.yticks(range(t), [f'T{i}' for i in range(t)], fontsize=20)
plt.xlabel('Bands and band related features', fontsize=20)
plt.ylabel('Time frames', fontsize=20)
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top') 

fig.subplots_adjust(wspace=0, hspace=0)

cb = fig.colorbar(im, ax=[ax], orientation='horizontal', pad=0.01, aspect=100)
cb.ax.tick_params(labelsize=20)
fig.savefig(f'figs/bands_and_features.png', bbox_inches='tight')

![image](figs/bands_and_features.png)